In [1]:
# !/usr/bin/env python
# coding: utf-8

import warnings
warnings.simplefilter("ignore")
from training_functions import *

A continuación se dará una breve explicación de lo que se ha hecho para optimizar el modelo elegido `HistGradientBoostingClassifier`.

Se ha hecho uso de muchos de los procedimientos comentados en la competencia de modelos (consultar __model_selection.ipynb__) pero, aquí, han sido generalizados vía funciones contenidas en el script __training_functions.py__. Cada una de estas funciones tiene una descripción de la labor que llevan a cabo. 

En aquel script, la función __optimization_model__ realiza la optimización del modelo.

# 1. ESPACIO HIPERPARAMETRAL.

El `HistGradientBoosting` es en esencia un boosting que considera árboles débiles que van aprediendo uno del otro de manera secuencial. De hecho, cada uno se enfoca en corregir el error del modelo anterior.

Este estimador es más rápido que un `GradientBoosting` clásico debido al uso del procesamiento paralelo en múltiples núcleos del CPU, además de considerar tratamiento de valores perdidos, regularización l2, restricciones de monotonía, entre otras cosas.

Se ha decidido trabajar con los siguientes hiperparámetros del estimador. Se dará una breve descripción de ellos.
1. __`max_iter`:__  Número de árboles (modelos débiles) que estarán involucrados en el boosting. \
Mientras mayor sea, mejor ajuste aunque puede dar paso al sobreajuste.
2. __`max_leaf_nodes`:__ Número máximo de hojas (nodos finales) en cada árbol individual. \
Controla la complejidad del modelo limitando la cantidad de divisiones en cada árbol.
3. __`min_samples_leaf`:__ Número mínimo de muestras en cada hoja. \
Un valor alto puede evitar divisiones que sean demasiado específicas evitando a su vez el sobreajuste.
4. __`learning_rate`:__ Determina la contribución de cada árbol en el proceso de boosting.\
Un valor pequeño da como resultado una convergencia más lenta pero un modelo más preciso, mientras que un valor grande puede conducir a un entrenamiento más rápido pero con mayor riesgo de sobreajuste.
5. __`max_depth`:__ Es la profundidad máxima permitida en los árboles. \
Ayuda a controlar la complejidad del modelo.
6. __`l2_regularization`__: Realiza una regularización tipo "Ridge".

Por supuesto, hay más hiperparámetros que podemos controlar. Sin embargo, decidimos trabajar con estos últimos por ser los más representativos al momento de construir árboles de decisión.

El __espacio hiperparametral__ está conformado por todas las combinaciones de valores de hiperparámetros, dado un rango de acción para cada uno de ellos. A continuación se exhibe la configuración del espacio hiperparametral que se ha usado en esta prueba.

In [2]:
space = {'max_iter': [200, 400, 650, 700, 800],
         'max_leaf_nodes': [10, 70, 100, 150], 
         'min_samples_leaf': [10, 20, 30, 40],
         'learning_rate':  [0.05, 0.07, 0.1, 0.3, 0.5, 1],
         'max_depth': [None, 1],
         'l2_regularization': [0]}

La elección de esta configuración es meramente arbitraria, aunque la idea es considerar tanto valores altos como valores bajos de los hiperparámetros. 

Dado que el espacio hiperparámetral es limitado, la función __optimization_model__ utiliza un `GridSearchCV`, el cual recorrerá todo el espacio y aplicará un __cross-validation__ por cada punto en ese espacio. El total de puntos que recorrerá es de 960. Si el espacio considerara rangos mucho más grandes de valores o bien considerara rangos continuos, entonces se debería utilizar un `RandomizedSearchCV` o un `BayesSearchCV`.

El sobreajuste del modelo se estará minitoriando al comparar las métricas de __cross-validation__ y las de __testeo__. Si éstas últimas sufren un deterioro importante, entonces habrá un serio problema de sobreajuste. Sin embargo, de acuerdo a experiencia propia, las métricas de __cross-validation__ suelen informar muy bien sobre la estabilidad general del modelo.

__Nota__: Aunque el modelo ha sido optimizado con el espacio hiperparametral anterior, en el script de funciones se ha dejado una versión más pequeña, esto para fines de prueba de código. Este espacio más reducido cuenta solo con 8 puntos de combinación.

# 2. CONFIGURACIONES GENERALES.

A continuación se explicará los valores de los parámetros con controlan la función __optimization_model__, aunque algunos de ellos ya han sido explicados a profundidad en la competencia de modelos. Estas explicaciones se darán en forma de comentarios de código python.

In [3]:
# Son los nombres de los binarios que contienen la data de entrenamiento y la data. Están en la carpeta outputs/.
name_train_sav='wA_train.sav'
name_test_sav='wA_test.sav'

In [4]:
# Lista de nombres de features y nombre de la variable objetivo.
features_names =  ['Location',
                   'MinTemp',
                   'Rainfall',
                   'WindGustDir',
                   'WindGustSpeed',
                   'WindDir9am',
                   'WindDir3pm',
                   'WindSpeed3pm',
                   'Humidity9am',
                   'Humidity3pm', 
                   'Pressure9am',
                   'RainToday', 
                   'month',
                   'week_of_year']
objective_name = 'RainTomorrow'

In [5]:
# Es el nombre del modelo ganador.
model_name = 'HistGradientBoostingClassifier'

In [6]:
# Es la semilla que gobernará los procesos.
seed = 5000
# Es la estrategia de submuestreo.
ratio_balance = 1
# Es el número de folds que conformarán el proceso de cross-validation.
k_folds = 4
# Un valor más grande hace más informativo el proceso de optimización.
verbose = 10

In [7]:
# Es la métrica de cross-validación que se optimizará. Por default es el f1.
# La combinación de hiperparámetros que obtenga el mayor valor de cv_f1 configurará el modelo optimizado final.
optimized_metric='cv_f1'

In [8]:
# Guarda en un binario la información del modelo optimizado.
save_best_info=False

In [9]:
# Ubicación de los binarios que contienen la data necesaria.
path='../outputs/'

# 3. OPTIMIZACIÓN Y GUARDADO DE INFORMACIÓN.

Dados los valores anteriores, procedemos a comenzar la optimización.

In [10]:
optim_results = optimization_model(name_train_sav=name_train_sav,
                                   name_test_sav=name_test_sav,
                                   features_names=features_names,
                                   objective_name=objective_name,
                                   model_name=model_name,
                                   seed=seed,
                                   ratio_balance=ratio_balance,
                                   k_folds=k_folds,
                                   verbose=verbose,
                                   optimized_metric=optimized_metric,
                                   save_best_info=save_best_info,
                                   path=path)

2023-07-04 06:52:49 INFO: PROCESO DE OPTIMIZACIÓN.
2023-07-04 06:52:49 INFO: CONFIGURACIÓN GENERALES.
2023-07-04 06:52:49 INFO: TUBERÍAS PARA PROCESAMIENTO Y MUESTREO.
2023-07-04 06:52:49 INFO: CONFIGURACIÓN PARA CV Y SEARCH.
2023-07-04 06:52:49 INFO: COMIENZA OPTIMIZACIÓN.


Fitting 4 folds for each of 8 candidates, totalling 32 fits
[CV 1/4; 1/8] START estimator__l2_regularization=0, estimator__learning_rate=0.5, estimator__max_depth=None, estimator__max_iter=200, estimator__max_leaf_nodes=10, estimator__min_samples_leaf=10, estimator__random_state=5000
[CV 1/4; 1/8] END estimator__l2_regularization=0, estimator__learning_rate=0.5, estimator__max_depth=None, estimator__max_iter=200, estimator__max_leaf_nodes=10, estimator__min_samples_leaf=10, estimator__random_state=5000; f1: (test=0.623) m_c: (test=0.504) precision: (test=0.528) recall: (test=0.761) total time=   0.9s
[CV 2/4; 1/8] START estimator__l2_regularization=0, estimator__learning_rate=0.5, estimator__max_depth=None, estimator__max_iter=200, estimator__max_leaf_nodes=10, estimator__min_samples_leaf=10, estimator__random_state=5000
[CV 2/4; 1/8] END estimator__l2_regularization=0, estimator__learning_rate=0.5, estimator__max_depth=None, estimator__max_iter=200, estimator__max_leaf_nodes=10, estim

[CV 4/4; 4/8] END estimator__l2_regularization=0, estimator__learning_rate=0.5, estimator__max_depth=None, estimator__max_iter=400, estimator__max_leaf_nodes=10, estimator__min_samples_leaf=20, estimator__random_state=5000; f1: (test=0.623) m_c: (test=0.503) precision: (test=0.523) recall: (test=0.769) total time=   1.0s
[CV 1/4; 5/8] START estimator__l2_regularization=0, estimator__learning_rate=0.5, estimator__max_depth=1, estimator__max_iter=200, estimator__max_leaf_nodes=10, estimator__min_samples_leaf=10, estimator__random_state=5000
[CV 1/4; 5/8] END estimator__l2_regularization=0, estimator__learning_rate=0.5, estimator__max_depth=1, estimator__max_iter=200, estimator__max_leaf_nodes=10, estimator__min_samples_leaf=10, estimator__random_state=5000; f1: (test=0.616) m_c: (test=0.494) precision: (test=0.522) recall: (test=0.750) total time=   1.1s
[CV 2/4; 5/8] START estimator__l2_regularization=0, estimator__learning_rate=0.5, estimator__max_depth=1, estimator__max_iter=200, esti

2023-07-04 06:53:24 INFO: CÁLCULO DE MÉTRICAS DE CV.
2023-07-04 06:53:24 INFO: SELECCIÓN DEL MEJOR MODELO VÍA UNA MÉTRICA DE CV.
2023-07-04 06:53:24 INFO: FIN DE OPTIMIZACIÓN.


[CV 4/4; 8/8] END estimator__l2_regularization=0, estimator__learning_rate=0.5, estimator__max_depth=1, estimator__max_iter=400, estimator__max_leaf_nodes=10, estimator__min_samples_leaf=20, estimator__random_state=5000; f1: (test=0.615) m_c: (test=0.493) precision: (test=0.519) recall: (test=0.755) total time=   1.1s


In [11]:
optim_results

hyperparameters      {'l2_regularization': 0, 'learning_rate': 0.5,...
cv_f1                                                         0.624172
cv_precision                                                  0.524443
cv_recall                                                     0.770797
cv_m_c                                                        0.505549
optimization_date                           2023-07-04 06:53:24.394637
ratio_balance                                                        1
model_name                              HistGradientBoostingClassifier
k_folds                                                              4
Name: 0, dtype: object

Al final, la función __optimization_model__ devuelve un panda-series con la información del modelo optimizado.

Se guardan la información del modelo optimizado.

In [12]:
# pickle.dump(optim_results, open('../outputs/optimization_results.sav', 'wb'))

# 4. COMPARACIÓN ENTRE EL MODELO OPTIMIZADO Y SIN OPTIMIZAR.

In [13]:
path='../outputs/'
optim_results = get_data(name_sav='optimization_results.sav', path=path)

La combinación ganadora de hiperparámetros es la siguiente.

In [14]:
optim_results['hyperparameters']

{'l2_regularization': 0,
 'learning_rate': 0.05,
 'max_depth': None,
 'max_iter': 800,
 'max_leaf_nodes': 70,
 'min_samples_leaf': 40,
 'random_state': 5000}

A simple vista, observamos que el modelo está requiriendo muchos estimadores débiles (__`max_iter`__=800).

Comparemos las métricas de __cross-validation__ del modelo optimizado con las del modelo no optimizado (ver __model_selection.ipynb__)

In [15]:
print('f1-cv optimizado', round(optim_results['cv_f1'], 2))
print('f1-cv NO optimizado', 0.63)
print('recall-cv optimizado', round(optim_results['cv_recall'], 2))
print('recall-cv NO optimizado', 0.78)
print('precision-cv optimizado', round(optim_results['cv_precision'], 2))
print('precision-cv NO optimizado', 0.53)
print('m_c-cv optimizado', round(optim_results['cv_m_c'], 2))
print('m_c-cv NO optimizado', 0.52)   

f1-cv optimizado 0.64
f1-cv NO optimizado 0.63
recall-cv optimizado 0.79
recall-cv NO optimizado 0.78
precision-cv optimizado 0.54
precision-cv NO optimizado 0.53
m_c-cv optimizado 0.53
m_c-cv NO optimizado 0.52


Vemos que solo se han conseguido mejoras de una centésima en todos las métricas de __cross-validation__.

Habrá que hacer un revisión desde el principio, con tal de encontrar mejoras contundentes. Los enfoques que se podrían llevar a cabo son:
1. Rescate, de alguna manera, de variables numéricas con porcentajes importantes de valores perdidos. En el análisis exploratorio se eliminaron algunas varibales y tal vez puedan ser añadidas al modelo al, por ejemplos, discretizarlas.
2. Hacer una ingeniería de variables más ambiciosa y rica. Aprovechar de manera más ingeniosa la variable de tiempo, por ejemplo.
3. Cambiar el estimador. El gran tamaño de la data nos da acceso a herramientas más complejas como redes neuronales.
4. Utilizar métodos de ensamblado de modelos.
5. Considerar otro enfoque de muestreo. Tal vez considerar técnicas como el SMOTE u otra ténica de sobremuestreo de la clase minoritaria.